In [16]:
import numpy as np
import pandas as pd

import re # regular expression
import os # operating system
import shutil # shell utilities
import fnmatch # file name matching
import warnings;warnings.filterwarnings("ignore")

Count lines like

defined( GEOS_FP ) || defined( MERRA2 ) 

or #if !defined( MERRA ) && !defined( GEOS_FP )

In [25]:
maindir='../Code_v11-02_all'

Tags = ['GEOS_FP','MERRA2','FLEXGRID']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  
n_tag = len(Tags)

fileFormat = '*.F*' # .F or .F90

data = [] # a list of list, to be converted to pandas DataFrame
for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        # KPP/*/gckpp_HetRates.F90 are mostly links, skip them
        if os.path.islink(filepath):
            continue
        
        # record current file, followed by n_tag+2 0s for later use
        data.append( [dirpath[len(maindir):],filename] + [0]*(n_tag+2) )
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            for line in file:
                for n,regex in enumerate(regex_list):          
                    if regex.match(line):
                        data[-1][n+2] += 1
                if regex_list[0].match(line) and regex_list[1].match(line):
                    if "||" in line:
                        data[-1][n_tag+2] += 1
                        #print(line)
                    if "&&" in line:
                        data[-1][n_tag+3] += 1
                        #print(line)

#label the raw data
df_raw = pd.DataFrame(data, columns=['dir','filename']+Tags+['FP || ME2','FP !&& ME2'])

#filter the files with zero match, and re-arrange the index
df = df_raw[(df_raw['GEOS_FP'] > 0) | (df_raw['MERRA2'] > 0)] 
df.reset_index(drop=True,inplace=True)

# add the counts of || and !&& 
df['oneline'] = df['FP || ME2'] + df['FP !&& ME2']
 
# display results
df[ (df["GEOS_FP"] != df["FLEXGRID"]) | (df["MERRA2"] != df["FLEXGRID"]) ]
#df

,dir,filename,GEOS_FP,MERRA2,FLEXGRID,FP || ME2,FP !&& ME2,oneline
2,/GeosCore,cleanup.F,1,1,0,0,0,0
13,/GeosCore,input_mod.F,5,6,4,2,2,4
15,/GeosCore,main.F,5,5,2,2,0,2
22,/GeosCore,tpcore_bc_mod.F,4,4,0,0,0,0
23,/GeosCore,tpcore_window_mod.F90,1,1,0,0,0,0
26,/GeosUtil,bpch2_mod.F,3,4,0,0,0,0
29,/GeosUtil,transfer_mod.F,6,6,5,5,0,5
30,/Headers,CMN_SIZE_mod.F,5,5,0,0,0,0
32,/HEMCO/Core,hco_extlist_mod.F90,1,1,0,0,0,0
35,/HEMCO/Extensions,hcox_dustdead_mod.F,3,3,0,0,0,0


Check the occurrence of Tags

Automatically change all lines including

defined( GEOS_FP ) || defined( MERRA2 )

to

defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

This should cover most of files. Manually change other codes.

- https://stackoverflow.com/questions/30319701/add-lines-to-text-file-after-occurence-of-certain-text

In [10]:
# warning: this cell will modify the code files!
# use git reset --hard to recover the orginal state

maindir='../Code_v11-02_all'

Tags = ['GEOS_FP','MERRA2']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  

fileFormat = '*.F*' # .F or .F90

for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        # KPP/*/gckpp_HetRates.F90 are mostly links
        if os.path.islink(filepath):
            continue    
    
        match = False # assume this file doest not contain the pattern
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            
            # read all lines, modify lines in-memory, and finally overwrite original file
            # this approach could be relative slow but clean.
            lines = file.readlines()
            
            for n,line in enumerate(lines):
                
                # the last character is always linebreak. Skip it.
                if regex_list[0].match(line) and regex_list[1].match(line):
                    if "||" in line:
                        print(filepath);print(line)
                        templine = line[:-1]+' || defined( FLEXGRID )\n'
                        print(templine)
                        lines[n]=templine
                        match = True
                    if "&&" in line:
                        print(filepath);print(line)
                        templine = line[:-1]+' && !defined( FLEXGRID )\n'
                        print(templine)
                        lines[n]=templine
                        match = True
                        
        # open the same file to overwrite
        if match:
            with open(filepath,"w",encoding = "ISO-8859-1") as file:
                for line in lines:
                    file.write(line)

../Code_v11-02_all/GeosCore/carbon_mod.F
#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../Code_v11-02_all/GeosCore/carbon_mod.F
#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if   defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../Code_v11-02_all/GeosCore/carbon_mod.F
#if   !defined( GEOS_5 ) && !defined( MERRA ) && !defined( GEOS_FP ) && !defined( MERRA2 )

#if   !defined( GEOS_5 ) && !defined( MERRA ) && !defined( GEOS_FP ) && !defined( MERRA2 ) && !defined( FLEXGRID )

../Code_v11-02_all/GeosCore/cldice_HBrHOBr_rxn.F
#if defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 )

#if defined( GEOS_5 ) || defined( MERRA ) || defined( GEOS_FP ) || defined( MERRA2 ) || defined( FLEXGRID )

../Code_v11-02_all/GeosCore